<a href="https://colab.research.google.com/github/hexagramg/iu5-BI/blob/master/lab/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:


import tensorflow.compat.v1 as tf
import numpy as np

tf.disable_v2_behavior()

In [0]:
bandits = [0.2,0,-0.2,-5]
num_bandits = len(bandits)
def pullBandit(bandit):
    
    result = np.random.randn(1)
    if result > bandit:
        #Выигрыш 
        return 1
    else:
        #Проигрыш
        return -1

In [0]:
tf.reset_default_graph()

#Часть сети прямого распространения. Осуществляет выбор.
weights = tf.Variable(tf.ones([num_bandits]))
chosen_action = tf.argmax(weights,0)

#Следующие строки осуществляют тренировку
reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
responsible_weight = tf.slice(weights,action_holder,[1])
loss = -(tf.log(responsible_weight)*reward_holder)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
update = optimizer.minimize(loss)

In [11]:
total_episodes = 1000 #Количество эпизодов
total_reward = np.zeros(num_bandits) #Таблица лидеров бандитов
e = 0.1 #Шанс случайного выбора

init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        
        if np.random.rand(1) < e:
            action = np.random.randint(num_bandits)
        else:
            action = sess.run(chosen_action)
        
        reward = pullBandit(bandits[action])
        
        #Обноаление сети
        _,resp,ww = sess.run([update,responsible_weight,weights], feed_dict={reward_holder:[reward],action_holder:[action]})
        
        #Обновление таблицы лидеров
        total_reward[action] += reward
        if i % 100 == 0:
            print(f"Награда для следующего количества бандитов {num_bandits}: {total_reward}")
        i+=1
print(f"Агент выбрал следующешл бандита лучшим -- {np.argmax(ww)+1}")
if np.argmax(ww) == np.argmax(-np.array(bandits)):
    print("...и он был прав")
else:
    print("...и он был не прав")

Награда для следующего количества бандитов 4: [-1.  0.  0.  0.]
Награда для следующего количества бандитов 4: [-2.  0.  0. 57.]
Награда для следующего количества бандитов 4: [ -2.   0.   0. 153.]
Награда для следующего количества бандитов 4: [  1.   0.   2. 244.]
Награда для следующего количества бандитов 4: [ -1.   0.   2. 338.]
Награда для следующего количества бандитов 4: [ -4.   1.   4. 428.]
Награда для следующего количества бандитов 4: [ -1.  -1.   4. 523.]
Награда для следующего количества бандитов 4: [ -1.  -3.   2. 617.]
Награда для следующего количества бандитов 4: [ -2.  -2.   3. 706.]
Награда для следующего количества бандитов 4: [ -1.   0.   4. 800.]
Агент выбрал следующешл бандита лучшим -- 4
...и он был прав
